In [ ]:
import zipfile
import os

# Unzip the dataset into the desired directory
os.makedirs("dataset", exist_ok=True)
with zipfile.ZipFile('/content/city-traffic-and-vehicle-behavior-dataset.zip', 'r') as zip_ref:
    zip_ref.extractall("dataset")

In [ ]:
import pandas as pd

df= pd.read_csv('/content/dataset/dt.csv')

In [ ]:
df.head()

,City,Vehicle Type,Weather,Economic Condition,Day Of Week,Hour Of Day,Speed,Is Peak Hour,Random Event Occurred,Energy Consumption,Traffic Density
0,New York,Car,Sunny,Stable,Monday,8,60,True,False,50,Medium
1,New York,Car,Sunny,Stable,Monday,12,55,True,False,48,High
2,New York,Bus,Rainy,Stable,Tuesday,16,40,False,False,60,High
3,Los Angeles,Car,Cloudy,Stable,Wednesday,10,65,False,False,45,Low
4,Los Angeles,Car,Sunny,Stable,Thursday,18,50,True,True,55,Medium


In [ ]:
Columns_to_drop = ['Economic Condition','Random Event Occurred','Energy Consumption','City']

In [ ]:
df = df.drop(columns=Columns_to_drop)

In [ ]:
df.count()

Vehicle Type       36
Weather            36
Day Of Week        36
Hour Of Day        36
Speed              36
Is Peak Hour       36
Traffic Density    36
dtype: int64

In [ ]:
df.head()


,Vehicle Type,Weather,Day Of Week,Hour Of Day,Speed,Is Peak Hour,Traffic Density
0,Car,Sunny,Monday,8,60,True,Medium
1,Car,Sunny,Monday,12,55,True,High
2,Bus,Rainy,Tuesday,16,40,False,High
3,Car,Cloudy,Wednesday,10,65,False,Low
4,Car,Sunny,Thursday,18,50,True,Medium


In [ ]:
weather_mapping = {'Sunny': 0, 'Rainy': 1, 'Cloudy': 2}
day_of_week_mapping = {'Monday': 0, 'Tuesday': 1, 'Wednesday': 2, 'Thursday': 3, 'Friday': 4, 'Saturday': 5, 'Sunday': 6}

df['Weather'] = df['Weather'].map(weather_mapping)
df['Day Of Week'] = df['Day Of Week'].map(day_of_week_mapping)

# Display the encoded DataFrame
print(df.head())

  Vehicle Type  Weather  Day Of Week  Hour Of Day  Speed  Is Peak Hour  \
0          Car      0.0            0            8     60          True   
1          Car      0.0            0           12     55          True   
2          Bus      1.0            1           16     40         False   
3          Car      2.0            2           10     65         False   
4          Car      0.0            3           18     50          True   

  Traffic Density  
0          Medium  
1            High  
2            High  
3             Low  
4          Medium  


In [ ]:
# Define mappings for the target variable 'Traffic Density'
traffic_density_mapping = {'Low': 0, 'Medium': 1, 'High': 2}

# Apply mapping to encode 'Traffic Density' into numerical values
df['Traffic Density'] = df['Traffic Density'].map(traffic_density_mapping)

# Display the encoded DataFrame
print(df.head())

  Vehicle Type  Weather  Day Of Week  Hour Of Day  Speed  Is Peak Hour  \
0          Car      0.0            0            8     60          True   
1          Car      0.0            0           12     55          True   
2          Bus      1.0            1           16     40         False   
3          Car      2.0            2           10     65         False   
4          Car      0.0            3           18     50          True   

   Traffic Density  
0                1  
1                2  
2                2  
3                0  
4                1  


In [ ]:
# Let's split our dataset into features (X) and target variable (y)
X = df.drop(['Traffic Density', 'Day Of Week', 'Weather','Vehicle Type'], axis=1)   # Features
y = df['Traffic Density']  # Target variable

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(random_state=42)

In [ ]:
# Next, let's train our model on the training data
model.fit(X_train, y_train)

# Once trained, let's make predictions on the test data
predictions = model.predict(X_test)

In [ ]:
# Now, let's evaluate our model's performance
from sklearn.metrics import classification_report, accuracy_score
print("Classification Report:")
print(classification_report(y_test, predictions))
print("Accuracy Score:", accuracy_score(y_test, predictions))

Classification Report:
              precision    recall  f1-score   support

        High       1.00      1.00      1.00         2
         Low       1.00      1.00      1.00         4
      Medium       1.00      1.00      1.00         2

    accuracy                           1.00         8
   macro avg       1.00      1.00      1.00         8
weighted avg       1.00      1.00      1.00         8

Accuracy Score: 1.0


In [ ]:
import joblib

# Define the file path to save the model
model_file_path = 'traffic_model.pkl'

# Save the trained model to the file path
joblib.dump(model, model_file_path)

print("Model saved successfully!")


Model saved successfully!


In [ ]:
# Assuming you have already trained the model and stored it in the variable 'model'

# Let's load new data for testing
new_data = {
    'Weather': ['Rainy', 'Rainy', 'Cloudy'],
    'Day Of Week': ['Monday', 'Monday', 'Tuesday'],
    'Hour Of Day': [16, 9, 16],
    'Speed': [10, 40, 100],
    'Is Peak Hour': [True, True, True]
}

new_df = pd.DataFrame(new_data)

# Apply the same mappings used for training data to encode the categorical variables
new_df['Weather'] = new_df['Weather'].map(weather_mapping)
new_df['Day Of Week'] = new_df['Day Of Week'].map(day_of_week_mapping)

# Make sure the columns are in the same order as the training data and contain all necessary features
expected_columns = X_train.columns
for column in expected_columns:
    if column not in new_df.columns:
        new_df[column] = 0  # Add missing columns with default value

# Reorder columns to match the order of the training data
new_df = new_df[expected_columns]

# Make predictions using the trained model
predictions = model.predict(new_df)

# Display the predictions
print("Predicted Traffic Densities:")
for i, pred in enumerate(predictions):
    print(f"Data point {i+1}: {pred}")


Predicted Traffic Densities:
Data point 1: 2
Data point 2: 1
Data point 3: 0


In [ ]:
# Load the pre-trained model
model = joblib.load('traffic_model.pkl')
